<a href="https://colab.research.google.com/github/KishenPatel97/TextClustering/blob/main/Initial_EDA_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [4]:
from datasets import load_dataset
import spacy

from pathlib import Path
from os import listdir
from os.path import isfile, join
import os

In [ ]:
# download stories from here
# https://cs.nyu.edu/~kcho/DMQA/

# 114 out of over 300,000 stories are missing text

# makes datafiles from this code
# https://github.com/becxer/cnn-dailymail/blob/master/make_datafiles.py

In [5]:
# """Load the dataset using huggingface data_loader"""
# type this command in the command line for the venv for this cell to work
# jupyter nbextension enable --py widgetsnbextension --sys-prefix 
dataset = load_dataset('cnn_dailymail', '3.0.0')

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
# CNN_STORIES_PATH = Path("D:/__School/__Masters/____2021fALL/5290 nlp/stories/cnn/stories")
# DM_STORIES_PATH = Path("D:/__School/__Masters/____2021fALL/5290 nlp/stories/dailymail/stories")

In [ ]:
# def grab_files(directory):
#     for name in os.listdir(directory):
#        full_path = os.path.join(directory, name)
#        if os.path.isdir(full_path):
#            for entry in grab_files(full_path):
#                yield entry
#        elif os.path.isfile(full_path):
#            yield full_path
#        else:
#            print('Unidentified name %s. It could be a symbolic link' % full_path)

In [ ]:
#i = 0
#for file in grab_files(CNN_STORIES_PATH):
#    i+=1
#    print(file)
#    if i == 5:
#        break

In [ ]:
#f = open(CNN_STORIES_PATH.joinpath("0001d1afc246a7964130f43ae940af6bc6c57f01.story"), "r")
#print(f.read())

In [ ]:
#f = open(CNN_STORIES_PATH.joinpath("0002095e55fcbd3a2f366d9bf92a95433dc305ef.story"), "r")
#print(f.read())